# How to use the Convenient Register Access Library

## Generate the python register model file

In [3]:
import sys
import numpy as np
from coreali import PythonExporter
from coreali import registerio
from systemrdl import RDLCompiler, RDLCompileError
from coreali.registerio import RegIoNoHW

input_files = ["../systemrdl/i2c_master_core.rdl",
"../systemrdl/logger.rdl",
"../systemrdl/top_sys.rdl"]
rdlc = RDLCompiler()

try:
    for input_file in input_files:
        rdlc.compile_file(input_file)
    root = rdlc.elaborate()
except RDLCompileError:
    sys.exit(1)

pythonExporter = PythonExporter()
pythonExporter.source_files = input_files
pythonExporter.export(root, "generated_regmodel.py")

## Create the register model object

In [4]:
import generated_regmodel
regmodel = generated_regmodel.top_sys(registerio.RegIoNoHW())
regmodel._rio.mem = np.zeros([1024], np.uint8)

## Print all registers and their content

In [5]:
print(regmodel)

top_sys               :
  Logger              :
    Ctrl              :          0 = 0x00000000
      ENABLE          :          0 = 0x00000000
      TRIGGERED       :          0 = 0x00000000
    Trig              : [0 0]
      SW_TRIG         : [0 0]
      LEVEL           : [0 0]
    LogMem            : [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
   ...
  I2c                 :
    PRERlo            :          0 = 0x00
      VAL             :          0 = 0x00
    PRERhi            :          0 = 0x00
      VAL             :          0 = 0x00
    CTR               :          0 = 0x00
      EN              :          0 = 0x00


## Get some help

In [6]:
regmodel.I2c.CTR.EN.help()

desc: EN, I2
C core enable bit.
When set to ‘1’, the core is enabled.
When set to ‘0’, the core is disabled.
sw: AccessType.r
hw: AccessType.r


In [7]:
## Access registers

In [8]:
regmodel.Logger.Ctrl.ENABLE.write(1) # set field to 1
regmodel.Logger.Trig[0:2].write([10, 20]) # write two fields at once

In [9]:
print(regmodel.Logger.Ctrl.ENABLE.read()) # read the value of field
print(regmodel.Logger.Trig.LEVEL.read()) # read some registers together

1
[ 5 10]
